In [42]:
import numpy as np

sequence = input().split()


# Encode bases as numbers and vice versa
bases_to_numbers = {'_':0, 'A':1, 'T':2, 'G':3, 'C':4, 'U':5}
numbers_to_bases = {0:'_', 1:'A', 2:'T', 3:'G', 4:'C', 5:'U'}

delta = float(sequence[2]) # gap penalty
mu = -1    # default mismatch penalty

# Transform sequence into numeric form
def btn(s):
    return np.array(list(map(lambda x: bases_to_numbers[x], list(s))))

weight_matrix_standard = np.array([[-1, delta, delta, delta, delta, delta],  #    gap A T G C U
                                  [delta, 1, mu, mu, mu, mu],             # gap
                                  [delta, mu, 1, mu, mu, mu],             # A 
                                  [delta, mu, mu, 1, mu, mu],             # T
                                  [delta, mu, mu, mu, 1, mu],             # G
                                  [delta, mu, mu, mu, mu, 1]])            # C
                                                                          # U
    
# Determine the shape of the input matrix and construct it in the same form as the standard one
m_size = int((len(sequence[3::]))**(1/2))
left_column = np.array([[delta]]*m_size)
top_row = np.array([[-1]] + [[delta]]*m_size).reshape(1, 1+m_size)
# The input matrix is weight_submatrix
weight_submatrix = np.array(list(map(float, sequence[3::]))).reshape((m_size, m_size))
weight_matrix_custom = np.r_[top_row, np.c_[left_column, weight_submatrix]]


# Get optimal alignment from optimal path
def path_to_alignment(seq1, seq2, path):
    seqA = list(seq1[::-1]) # We move from the beginning of the sequence to the end
    seqB = list(seq2[::-1]) # and .pop() gives us the last element, so we reverse the order
    path = path[::-1]       # path is generated from the end to the beginning, so we reverse it too
    aligned_seq1, aligned_seq2  = [], []
    
    def diff(i): # displacement vector at each step. Tells us what pair of symbols to choose
        return np.array(path[i+1]) - np.array(path[i])
    
    diff_path = list(map(diff, range(len(path)-1))) # rewrite the path as a sequence of displacements
    for i in range(len(diff_path)): # if (1, 1), append two nucleotides, 
                                    # if (0, 1) or (1, 0), append gap+nucl or vice versa
        A = numbers_to_bases[seqA.pop()] if diff_path[i][0] == 1 else '_' 
        B = numbers_to_bases[seqB.pop()] if diff_path[i][1] == 1 else '_'
        aligned_seq1.append(A)
        aligned_seq2.append(B)
        
    return ''.join(aligned_seq1), ''.join(aligned_seq2)

# The algorithm 
def needleman_wunsch(seq1, seq2, weight_matrix):
    len1, len2 = len(seq1), len(seq2)
    
    paths = {} # For each element of the path scores matrix we will determine and remember 
               # the optimal precedent element
               # at the same time as we determine the value of the element itslef. paths{} is a
               # dictionary of links: path : (i, j) -> optimal_precedent((i, j))
    path_scores = np.zeros([len1+1, len2+1]) #Path scores matrix initialization
    path_scores[0, 0] = 0
    path_scores[1:, 0] = [delta*i for i in range(1, len1+1)]
    path_scores[0, 1:] = [delta*j for j in range(1, len2+1)]
    for i in range(1, len1+1): # Optimal precedent elements on the boundary are determined uniquely
        paths[(i, 0)] = (i-1, 0)
    for j in range(1, len2+1):
        paths[(0, j)] = (0, j-1)
                        
        
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            prev_pos = ((i-1, j-1), (i, j-1), (i-1, j)) # Possible precedent elements
            # Scores of paths that come from the set of optimal precedent elements
            prev_scores = (path_scores[i-1, j-1] + weight_matrix[seq1[i-1], seq2[j-1]],
                                    path_scores[i, j-1] + delta, 
                                    path_scores[i-1, j] + delta)
                                    
            path_scores[i, j] = max(prev_scores)
            # Add the link to the optimal precedent element into paths{}
            paths[(i, j)] = max(zip(prev_pos, prev_scores), key = lambda x: x[1])[0]
        
    optimal_path = [(len1, len2)] # optimal path must contain the element at the lower-right angle
    
    while optimal_path[-1] in paths: # Follow the links to the optimal elements backwards 
                                     # until we reach the element (0, 0)
        optimal_path.append(paths[optimal_path[-1]])
    
    return seq1, seq2, path_scores, optimal_path, path_to_alignment(seq1, seq2, optimal_path)

print(*needleman_wunsch(btn(sequence[0]), btn(sequence[1]), weight_matrix_custom)[4])

AAAA AATA -1 1 0 -1 -1 0 1 -1 -1 -1 -1 1 -1 -1 -1 -1 1
AAAA AATA


In [130]:


#print(btn(test_seq_1))
#print(btn(test_seq_2))


#weight_matrix_2 = np.array([[-1,-2,-2,-2,-2],
#                            [-2, 2,-1, 1,-1],
#                            [-2,-1, 2,-1, 1],
#                            [-2, 1,-1, 2,-1],
#                            [-2,-1, 1,-1, 2]])


[2 3 2 2 1 4 3 2 1]
[3 3 2 2 3 3]


In [39]:
a = np.array([[1]]*3)
b = np.array([[-1]] + [[1]]*3).reshape((1, 4))
c = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])


np.r_[b, np.c_[a, c]]


array([[-1,  1,  1,  1],
       [ 1,  1,  0,  0],
       [ 1,  0,  1,  0],
       [ 1,  0,  0,  1]])

In [134]:
        
#test_path = [(9, 6),
#             (8, 5),
#             (7, 5),
#             (6, 4),
#             (5, 4),
#             (4, 4),
#             (3, 3),
#             (2, 2),
#             (1, 1),
#             (0, 0)]

#path_to_alignment(btn(test_seq_1), btn(test_seq_2), test_path)

('TGTTACGTA', 'GGTT__G_G')

In [136]:


needleman_wunsch(btn(test_seq_1), btn(test_seq_2), weight_matrix_2)

(array([2, 3, 2, 2, 1, 4, 3, 2, 1]),
 array([3, 3, 2, 2, 3, 3]),
 array([[  0.,  -2.,  -4.,  -6.,  -8., -10., -12.],
        [ -2.,  -1.,  -3.,  -2.,  -4.,  -6.,  -8.],
        [ -4.,   0.,   1.,  -1.,  -3.,  -2.,  -4.],
        [ -6.,  -2.,  -1.,   3.,   1.,  -1.,  -3.],
        [ -8.,  -4.,  -3.,   1.,   5.,   3.,   1.],
        [-10.,  -6.,  -3.,  -1.,   3.,   6.,   4.],
        [-12.,  -8.,  -5.,  -2.,   1.,   4.,   5.],
        [-14., -10.,  -6.,  -4.,  -1.,   3.,   6.],
        [-16., -12.,  -8.,  -4.,  -2.,   1.,   4.],
        [-18., -14., -10.,  -6.,  -4.,  -1.,   2.]]),
 [(9, 6),
  (8, 5),
  (7, 5),
  (6, 4),
  (5, 4),
  (4, 4),
  (3, 3),
  (2, 2),
  (1, 1),
  (0, 0)],
 ('TGTTACGTA', 'GGTT__G_G'))

In [15]:
a = (1, 2, 3)
b = (2, 8, 7)

i = 5
j = 8

print(list(zip(a, b)))
print(max(zip(a, b), key = lambda x: x[1]))
d = {}

for i in range(10):
    d[(i, 0)] = (i-1, 0) 
    

c = [1, 2, 3]
print(c[::-1].pop())
print(c[::-1].pop())
print(d)

[(1, 2), (2, 8), (3, 7)]
(2, 8)
1
1
{(0, 0): (-1, 0), (1, 0): (0, 0), (2, 0): (1, 0), (3, 0): (2, 0), (4, 0): (3, 0), (5, 0): (4, 0), (6, 0): (5, 0), (7, 0): (6, 0), (8, 0): (7, 0), (9, 0): (8, 0)}
